# Setup

In [3]:
"""
Page setup and execution for the aerospace mechanism chatbot
Example :        
-What can you tell me about latch mechanism design failures which have occurred        
-Follow up: Which one of the sources discussed volatile spherical joint interfaces           
"""
# import databutton as db

import os
import glob
import data_import
import queries
import pinecone
from dotenv import load_dotenv, find_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
import openai
import secrets

from IPython.display import display, Markdown
import pprint

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))
# print(os.getenv('LANGCHAIN_TRACING_V2'))
# print(os.getenv('LANGCHAIN_ENDPOINT'))
# print(os.getenv('LANGCHAIN_API_KEY'))
# print(os.getenv('LANGCHAIN_PROJECT'))

## Tradeable fields

In [4]:
output_level=None   # Max token limit, see section below
k=10 # Number of queries to return
search_type='similarity'    #  'mmr' or 'similarity'
temperature=0
verbose=True
chain_type='stuff'  # 'stuff' or  'map_reduce'
index_name = 'canopy--ams'

In [5]:
# Process some items
if output_level == "Concise":
    max_generated_tokens = 50
elif output_level == "Detailed":
    max_generated_tokens = 516
else:
    max_generated_tokens = -1

# Track filtering, set history to be blank
message_id=0
filter_toggle=False # Filter sources on last answer
chat_history = []

## Vector database

In [6]:
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT')
)

## Embeddings

In [7]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

## LLM

In [8]:
model_name='gpt-3.5-turbo-instruct'

llm = OpenAI(temperature=temperature,
             model_name=model_name,
             max_tokens=max_generated_tokens,
             tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)])

# Prompts

In [9]:
prompt=['Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?',
        'Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?']

## First prompt

In [11]:
# Set up object
tags=['prompt1']
qa_model_obj=queries.QA_Model(index_name,
                    embeddings_model,
                    llm,
                    k,
                    search_type,
                    verbose,
                    filter_arg=filter_toggle)

# Generate a response using your chat model
qa_model_obj.query_docs(prompt[0],tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown((prompt[0])))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?

 The types of lubricants to be avoided when designing space mechanisms are silicone lubricants, perfluoropolyether (PFPE) lubricants, and dry film lubricants. 
SOURCES: AMS_1998.pdf, AMS_2020.pdf, AMS_2020.pdf, AMS_2020.pdf, AMS_2020.pdf, AMS_1997.pdf, AMS_2004.pdf, AMS_2020.pdf, AMS_2018.pdf, AMS_2018.pdf

[{'document_id': 'AMS_1998.pdf_31107727471-051f-4979-8b7b-ff5aafb8233d', 'page': '311', 'source': 'AMS_1998.pdf'},
 {'document_id': 'AMS_2020.pdf_2684dd10ab7-7371-4b76-9347-8e22cb98ea41', 'page': '268', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_2655d737dec-85ef-4c92-a748-5606ab9cf2bc', 'page': '265', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_308b4ceecb7-7783-48c3-9975-62fd28ebe114', 'page': '308', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_297f3e14198-e3a0-4689-989e-a3dee241f0ae', 'page': '297', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_1997.pdf_289ab65038c-470e-4a9f-b81f-bc837a90ff37', 'page': '289', 'source': 'AMS_1997.pdf'},
 {'document_id': 'AMS_2004.pdf_288186c6849-fdc1-4229-b4af-9017670e8110', 'page': '288', 'source': 'AMS_2004.pdf'},
 {'document_id': 'AMS_2020.pdf_52786cdb6fb-8d6a-444e-a0db-b1ed3e0f2602', 'page': '527', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2018.pdf_90422f64ac-3adc-48fa-adf3-44bbf647e037', 'page': 

## Second Prompt

In [12]:
# Generate a response using your chat model
tags=['prompt2_nofilter']
qa_model_obj.update_model(llm=llm,
                          filter_arg=False)

qa_model_obj.query_docs(prompt[1],
                        tags=tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown((prompt[1])))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?

 Failures have occurred due to lubricant starvation and depletion, as well as challenges with finding effective additives. 
SOURCES: AMS_2020.pdf, AMS_2018.pdf

[{'document_id': 'AMS_2020.pdf_2684dd10ab7-7371-4b76-9347-8e22cb98ea41', 'page': '268', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_297f3e14198-e3a0-4689-989e-a3dee241f0ae', 'page': '297', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_314f564551c-6e39-4ac2-b21c-af1d540e1150', 'page': '314', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_308b4ceecb7-7783-48c3-9975-62fd28ebe114', 'page': '308', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_2655d737dec-85ef-4c92-a748-5606ab9cf2bc', 'page': '265', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2018.pdf_90422f64ac-3adc-48fa-adf3-44bbf647e037', 'page': '90', 'source': 'AMS_2018.pdf'}]


# Langsmith

In [11]:
# from langchain.chat_models import ChatOpenAI

# llm=ChatOpenAI()
# llm.invoke("Hello world!")